In [1]:
from judge_utils import *


/home/r3lativo/miniconda3/envs/mllms-con/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results_data = load_all_results(results_path)

In [3]:
def analyze_run(cmd, json_file_name):
  json_path = os.path.join(results_path, json_file_name)
  parsed_conversation, builder_actions = extract_conversation_data(json_path)
  evaluation = ask_judge(cmd, parsed_conversation)
  try:
    return int(evaluation)
  except ValueError:
    return "Not a number"
  

In [4]:
results_data["BASE"] = results_data["json_file"].apply(lambda x: analyze_run("BASE", x))

In [5]:
results_data["CLAR_Q"] = results_data["json_file"].apply(lambda x: analyze_run("CLAR_Q", x))

In [6]:
results_data["COMM_SH_REF"] = results_data["json_file"].apply(lambda x: analyze_run("COMM_SH_REF", x))

In [7]:
results_data["IMPL_REF"] = results_data["json_file"].apply(lambda x: analyze_run("IMPL_REF", x))

In [8]:
results_data

,run_time,structure_name,json_file,Model,Quantization,Device,Number of models,Max new tokens,Repetition Penalty,Max rounds,use_img,use_json,num_rounds,total_round_time_min,finished_by_architect,BASE,CLAR_Q,COMM_SH_REF,IMPL_REF
0,2025-02-12-1445-39,C15_double_stairs,2025-02-12-1445-39_C15_double_stairs.json,llava-hf/llava-v1.6-mistral-7b-hf,4-bit,cuda,2,2048,1.1,50,True,True,17,104.0,False,Not a number,Not a number,Not a number,Not a number
1,2025-02-11-1829-56,C1_bell,2025-02-11-1829-56_C1_bell.json,llava-hf/llava-v1.6-mistral-7b-hf,4-bit,cuda,2,2048,1.1,50,True,False,5,1.0,True,2,Not a number,Not a number,Not a number
